In [2]:
#Bibliotecas
from google.cloud import bigquery
import pandas as pd
import os
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [3]:
#Configura variable de entorno
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "prctica-etl-2af93f84aca1.json"#Credencial

In [4]:
client = bigquery.Client(location="southamerica-west1", project="prctica-etl")#Acceso al cliente de Big Query

In [5]:
query = "SELECT * FROM `prctica-etl.Yelp.business`"
df1 = client.query(query).to_dataframe()
query = "SELECT * FROM `prctica-etl.Yelp.Reviews`"
df2 = client.query(query).to_dataframe()

In [6]:
df_business = df1.copy()
df_reviews = df2.copy()

In [7]:

def user_recommender(business_id= str):#Ingresa el id de negocio calificado por el usuario
    umbral_calificacion = 3 #Se considera bien calificado al local con un puntaje mayor a 3
    rango_error = 2 #Rango error latitud
    local_calificado = df_business[df_business.business_id == business_id]#local de referencia del usuario
    reviews = df_reviews[['business_id','stars','useful', 'funny', 'cool', 'text', 'feeling']].copy()#Accede a las columnas de reviews
    reviews = pd.merge(df_reviews, df_business[['business_id', 'city', 'state', 'categories', 'latitude', 'longitude']], how='right',on='business_id')#Se hace un merge de los datos de reviews con datos relevantes de business para el modelo de ML
    review_max = reviews[reviews.business_id == business_id]#Selecciona las reviews del local
    review_max = review_max.sort_values(by='stars', ascending=False)#Las ordena con el fin de acceder a la mas alta
    reviews = reviews[reviews.state == local_calificado.state.iloc[0]]#Selecciona locales del mismo estado
    reviews = reviews[(reviews.latitude.astype(float) >=
                                                float(local_calificado.latitude.iloc[0]) - rango_error) & 
                                                (reviews.latitude.astype(float) <= 
                                                 float(local_calificado.latitude.iloc[0]) + rango_error)]
    reviews = reviews[(reviews.longitude.astype(float) >=
                                                float(local_calificado.longitude.iloc[0]) - rango_error) & 
                                                (reviews.longitude.astype(float) <= 
                                                 float(local_calificado.longitude.iloc[0]) + rango_error)]
    reviews = reviews[reviews.business_id != business_id]#Selecciona locales del mismo estado
    reviews = pd.concat([review_max.iloc[0].to_frame().T, reviews], ignore_index=True)#La review mas alta del local se ubica en la primera fila del df
    reviews = reviews.head(5000)#Limita el df a 5000 entradas (Se puede limitar en la query de acceso)
    reviews = reviews.astype(str)#Transforma todos los datos del df a str

    #Vectorizar todo el df por filas
    vectorizer = TfidfVectorizer(binary=True)
    X = vectorizer.fit_transform(reviews.apply(lambda x: ' '.join(x), axis=1))
    columnas = vectorizer.get_feature_names_out()
    reviews_vectorizado = pd.DataFrame(X.toarray(), columns=columnas)

    #Calcular similitud entre la fila 0 (nuestra unidad de referencia) y el resto de las filas
    referencia = reviews_vectorizado.iloc[0].values.reshape(1, -1)
    similitud = cosine_similarity(referencia, reviews_vectorizado)
    similitud_serie = pd.Series(similitud[0], index=reviews.index)#Se le asignan los indices de el df original a la matriz de similitudes
    resultado = reviews.loc[similitud_serie.nlargest(50).index]#Se seleccionan los primeros resultados (Margen para trabajar resultados de locales repetidos), la primera fila siempre es la de referencia
    
    #Armado de resultado
    id_array = resultado.business_id.iloc[1:]#Se obtiene un array con los 'business_id' del resultado ignorando la posicion 0 (referencia)
    filas_seleccionadas = df_business[df_business.business_id.isin(id_array)]#Se seleccionan las filas del df business que coincidan con los ID
    filas_seleccionadas = filas_seleccionadas[filas_seleccionadas.stars.astype(float) > umbral_calificacion]#Se seleccionan las filas cuyo puntaje 'stars' este por encima del umbral (para ignorar locales con baja calificacion)
    

    return filas_seleccionadas.head() #devuelve el df con los primeros 5 resultados

user_recommender('TvnqERlxc8lFnpERkP8JCg')#Prueba con un ID de df_business al azar

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,categories,platform
3,Rag7H_6BDv9SBoMZxLqD-w,Ten Asian Food Hall,"706 Haddonfield Rd, Ste 5",Cherry Hill,CA,08002,39.9297617,-75.0314522,4.0,96,"Restaurants, Poke, Asian Fusion, Food, Bubble Tea",Yelp
6,15hYNwnUrZ2zdjYunDE8lw,PDQ Cherry Hill,614 Haddonfield Rd,Cherry Hill,CA,08002,39.9312573,-75.0311541,3.5,160,"Sandwiches, American (Traditional), Restaurant...",Yelp
8,nQApvLYqMHEhbihbmPdngQ,Iceland,"1401 Little Gloucester Rd, Unit 8",Blackwood,CA,08012,39.7929702221,-75.0385532901,4.5,25,"Food, Juice Bars & Smoothies, Bubble Tea, Dess...",Yelp
16,6HUagGJpRoBo5FOIY8v2tA,The Bread Board Plus,605 N Haddon Ave,Haddonfield,CA,08033,39.9043662749,-75.0409501552,4.5,111,"Sandwiches, Restaurants, Salad, Breakfast & Br...",Yelp
19,183RrwDm8mmdKCX1E9dUVQ,K & A Bagel Cafe,1426 NJ-70,Cherry Hill,CA,08034,39.9112057,-74.9920186,4.5,177,"Breakfast & Brunch, Restaurants, Sandwiches, F...",Yelp
